In [8]:
import pandas as pd
import csv
import numpy as np

In [9]:
f = open('../Data/Preproceso/storms_final_preproceso.csv')
csv_f = csv.reader(f)

In [10]:
cabeceras = csv_f.next()

In [11]:
cabeceras

['EVENT_ID',
 'BEGIN_DATE_TIME',
 'BEGIN_LAT',
 'BEGIN_LON',
 'DAMAGE_CROPS',
 'DAMAGE_PROPERTY',
 'DEATHS_DIRECT',
 'DEATHS_INDIRECT',
 'END_DATE_TIME',
 'END_LAT',
 'END_LON',
 'EPISODE_ID',
 'EVENT_TYPE',
 'INJURIES_DIRECT',
 'INJURIES_INDIRECT',
 'MAGNITUDE',
 'RANGE',
 'TOR_F_SCALE']

In [12]:
tabla_pre = dict()

for row in csv_f:
    ep = dict(zip(cabeceras, row))
    if row[11] in tabla_pre:
        tabla_pre[row[11]].append(ep)
    else:
        tabla_pre[row[11]] = [ep]

In [13]:
tabla_trans = []

for ep_id in tabla_pre:
    ep_trans = {'EPISODE_ID':  0,
                'BEGIN_DATE' : 0,
                'END_DATE' :   0,
                'LAT' :        0,
                'LON' :        0,
                'RANGE':       0,
                'LLUVIA':      0,
                'VIENTO':      0,
                'TORNADO':     0,
                'GRANIZO':     0,
                'NIEVE':       0,
                'T. ELECTRICA':0                 
    }
    
    episodio = tabla_pre[ep_id]
    for evento in episodio: 
        if ep_trans['BEGIN_DATE'] == 0:
            ep_trans['BEGIN_DATE'] = np.datetime64(evento['BEGIN_DATE_TIME'])
        elif ep_trans['BEGIN_DATE'] >= np.datetime64(evento['BEGIN_DATE_TIME']):
            ep_trans['BEGIN_DATE'] = np.datetime64(evento['BEGIN_DATE_TIME'])
            
        if ep_trans['END_DATE'] == 0:
            ep_trans['END_DATE'] = np.datetime64(evento['END_DATE_TIME'])
        elif ep_trans['END_DATE'] <= np.datetime64(evento['END_DATE_TIME']):
            ep_trans['END_DATE'] = np.datetime64(evento['END_DATE_TIME'])
        
        if evento['EVENT_TYPE'] == 'Heavy Rain' and ep_trans['LLUVIA'] == 0:
            ep_trans['LLUVIA'] = 1
        elif evento['EVENT_TYPE'] == 'Flood' and ep_trans['LLUVIA'] <= 1:
            ep_trans['LLUVIA'] = 2
        elif evento['EVENT_TYPE'] == 'Flash Flood' and ep_trans['LLUVIA'] <= 2:
            ep_trans['LLUVIA'] = 3
        elif evento['EVENT_TYPE'] == 'Strong Wind' and ep_trans['VIENTO'] <= evento['MAGNITUDE']:
            ep_trans['VIENTO'] = 1
        elif evento['EVENT_TYPE'] == 'High Wind' and ep_trans['VIENTO'] <= evento['MAGNITUDE']:
            ep_trans['VIENTO'] = 2
        elif evento['EVENT_TYPE'] == 'Thunderstorm Wind' and ep_trans['VIENTO'] <= evento['MAGNITUDE']:
            ep_trans['VIENTO'] = 3
        elif evento['EVENT_TYPE'] == 'Tornado':
            ep_trans['TORNADO'] = int(evento['TOR_F_SCALE'][-1]) + 1 
        elif evento['EVENT_TYPE'] == 'Hail' and ep_trans['GRANIZO'] < evento['MAGNITUDE']:
            ep_trans['GRANIZO'] = evento['MAGNITUDE']
        elif evento['EVENT_TYPE'] == 'Winter Storm':
            ep_trans['NIEVE'] = 1
        elif evento['EVENT_TYPE'] == 'Heavy Snow':
            ep_trans['NIEVE'] = 1
        elif evento['EVENT_TYPE'] == 'Lightning':
            evento['T. ELECTRICA'] = 1
        else:
            #print evento['EVENT_TYPE']
            pass
        ep_trans['EPISODE_ID'] = ep_id
    tabla_trans.append(ep_trans)
        
        

/home/neverhope/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:20: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
/home/neverhope/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:25: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.


In [14]:
df = pd.DataFrame(tabla_trans)
df = df.set_index('EPISODE_ID')
df.to_csv('../Data/Transformacion/transformacion.csv')
df

,BEGIN_DATE,END_DATE,GRANIZO,LAT,LLUVIA,LON,NIEVE,RANGE,T. ELECTRICA,TORNADO,VIENTO
EPISODE_ID,,,,,,,,,,,
73398,2013-06-01 00:04:00,2013-06-01 05:45:00,0,0,3,0,0,0,0,0,3
65098,2012-06-29 15:06:00,2012-06-29 16:04:00,0,0,0,0,0,0,0,0,3
4025,2007-05-15 13:50:00,2007-05-15 17:41:00,0,0,0,0,0,0,0,1,3
22667,2008-09-12 18:25:00,2008-09-12 18:30:00,0,0,0,0,0,0,0,0,3
22660,2008-09-08 21:04:00,2008-09-09 02:02:00,1.75,0,0,0,0,0,0,0,3
61456,2012-03-02 13:10:00,2012-03-02 14:11:00,0.88,0,0,0,0,0,0,0,0
14290,2008-02-05 22:40:00,2008-02-06 04:30:00,1.0,0,3,0,0,0,0,0,3
1211378,2006-05-10 17:22:00,2006-05-10 17:27:00,0,0,0,0,0,0,0,1,0
73088,2013-04-19 05:00:00,2013-04-30 23:59:00,0,0,2,0,0,0,0,0,0
